# Heroes of Pymoli Data Anaysis

The overwhelming majority users are male, and the overwhelming majority of purchases comes from users aged 19-26, so it would be wise to market heavily towards the college/young-adult male crowd.

While the Top 5 Spenders bought at least three items, a majority bought only one. Introducing new items, or the increasing their relative power boost in-game, could encourage more purchases from the one-time shoppers

While the Top 5 Sellers are mostly under three dollars, the Top 5 Most Profitable items include three over four dollars. It would be wise to continue offering a selection of high-cost/high-value items for those who will pay, along with a large selection of low-cost items to suit the rest.

I noticed that some of the top-grossing items had been sold at lower prices, which led me to taking the average price sold at instead of using the max to simply return what I assumed would be the same value down the column. My comment about this would be it might be better to maintain exclusivity of high-cost items by not offering “sales” and instead introduce another, less-powerful option. If the people willing to pay for the premium items feel like they didn’t actually need to, they might be discouraged from future premium releases. 


In [1]:
# DEPENDENCIES
import pandas as pd
import numpy as np
import os

In [2]:
# Call the JSON
pymoli = pd.read_json("purchase_data.json")

# Player Count

In [3]:
# Find the Total Number of Players from the length of the unique SN list
total_players = len(pymoli["SN"].unique())
total_players_df = pd.DataFrame([{"Total Players":total_players}])
total_players_df

,Total Players
0,573


# Purchasing Analysis (Total)

In [4]:
# Find the Total Unique Items, Avg Price, Total Purchases, and Total Revenue

total_purch = pymoli["Price"].count()
unique_items = len(pymoli["Item Name"].unique())
avg_price = round(pymoli["Price"].mean(),2)
total_rev = round(float(pymoli["Price"].sum()),2)

summary_df = pd.DataFrame([{"Total Purchases":total_purch,
                            "Unique Items":unique_items,
                            "Average Price":avg_price,
                            "Total Revenue":total_rev}])

summary_df = summary_df[['Unique Items','Average Price','Total Purchases','Total Revenue']]
summary_df

,Unique Items,Average Price,Total Purchases,Total Revenue
0,179,2.93,780,2286.33


# Gender Demographics

In [5]:
# Drop duplicate SN entries to get unique players
unique_players = pymoli.drop_duplicates(["SN"],keep="first")

# Create df with the totals per gender, rename column
gender_count = pd.DataFrame(unique_players["Gender"].value_counts())
gender_count.columns = ["Total Count"]

# Set total users, Add column for percentage
total_users = gender_count["Total Count"].sum()
gender_count["Percentage"] = [round((x/total_users)*100) for x in gender_count["Total Count"]]

gender_count

,Total Count,Percentage
Male,465,81.0
Female,100,17.0
Other / Non-Disclosed,8,1.0


# Purchasing Analysis (Gender)

In [6]:
# Find Purchase Count by Gender
male_purchases = pymoli.loc[pymoli["Gender"]=="Male",["Price"]]
female_purchases = pymoli.loc[pymoli["Gender"]=="Female",["Price"]]
other_purchases = pymoli.loc[pymoli["Gender"]=="Other / Non-Disclosed",["Price"]]

# Set the values for Total, Average, and Count of purchases per gender value
total_m_purchase = round(float(male_purchases.sum()),2)
total_f_purchase = round(float(female_purchases.sum()),2)
total_o_purchase = round(float(other_purchases.sum()),2)

avg_m_purchase = round(float(male_purchases.mean()),2)
avg_f_purchase = round(float(female_purchases.mean()),2)
avg_o_purchase = round(float(other_purchases.mean()),2)

num_m_purchase = round(int(male_purchases.count()),2)
num_f_purchase = round(int(female_purchases.count()),2)
num_o_purchase = round(int(other_purchases.count()),2)

# Copy the previous dataframe to retain formatting
gender_summary = gender_count

# Create new columns based on Purchase Count, Avg Purchase, and Total Value
gender_summary["Purchase Count"] = [num_m_purchase,num_f_purchase,num_o_purchase]
gender_summary["Average Purchase"] = [avg_m_purchase,avg_f_purchase,avg_o_purchase]
gender_summary["Total Value"] = [total_m_purchase,total_f_purchase,total_o_purchase]

# Create a Normalized Value column
# (This doesn't appear to be the same type of value as in the example, but I don't know any other meaning for "normalize" other than to make them equal 1.)
total_value = gender_summary["Total Value"].sum()
gender_summary["Normalized Total"] = [round((x/total_value),2) for x in gender_summary["Total Value"]]

# Remove columns from previous exercise
gender_summary = gender_summary[["Purchase Count","Average Purchase","Total Value","Normalized Total"]]
gender_summary

,Purchase Count,Average Purchase,Total Value,Normalized Total
Male,633,2.95,1867.68,0.82
Female,136,2.82,382.91,0.17
Other / Non-Disclosed,11,3.25,35.74,0.02


# Age Demographics

In [7]:
# Identify the lowest and highest age in the df
low = pymoli["Age"].min()
high = pymoli["Age"].max()

# Create a list from low to high in intervals of 4
age_bins = np.arange((low-1),high,4)

# Create a list of labels to associate with the bins
bin_labels = ["10 & Under","11-14","15-18","19-22","23-26","27-30","31-34","35-38","39+"]

# Create new column for bins
pymoli["Age Group"] = pd.cut(pymoli["Age"],age_bins,labels=bin_labels)

# Create GroupBy object
age_binned = pymoli.groupby(["Age Group"])

# Create a dataframe from the count of the Age column
age_binned_df = pd.DataFrame(age_binned["Age"].count())

# Rename Total column and Add Percentage column
age_binned_df.columns = ['Total Players']
age_binned_df["Percentage"] = [round(((x/total_players)*100),2) for x in age_binned_df["Total Players"]]
age_binned_df

,Total Players,Percentage
Age Group,,
10 & Under,32,5.58
11-14,31,5.41
15-18,111,19.37
19-22,231,40.31
23-26,207,36.13
27-30,63,10.99
31-34,46,8.03
35-38,37,6.46
39+,20,3.49


# Purchasing Analysis (Age)

In [12]:
# Find Total Purchase Count by Age Group
count_binned = pd.DataFrame(age_binned["Price"].count())
count_binned.columns = ["Purchase Count"]
count_binned.reset_index(inplace=True)

# Find Average Purchase Price by Age Group
average_binned = pd.DataFrame(round(age_binned["Price"].mean(),2))
average_binned.columns = ["Average Purchase"]
average_binned.reset_index(inplace=True)

# Find Total Amount spent by Age Group
sum_binned = pd.DataFrame(round(age_binned["Price"].sum(),2))
sum_binned.columns = ['Total Spent']
sum_binned.reset_index(inplace=True)

# Merge dfs
merged_bins = pd.merge(count_binned,average_binned,on="Age Group")
merged_bins = pd.merge(merged_bins,sum_binned,on="Age Group")

# Add a Normalized Total column
total_sales = total_m_purchase + total_f_purchase + total_o_purchase
merged_bins["Normalized Total"] = [round((x/total_sales),2) for x in merged_bins["Total Spent"]]

merged_bins

,Age Group,Purchase Count,Average Purchase,Total Spent,Normalized Total
0,10 & Under,32,3.02,96.62,0.04
1,11-14,31,2.70,83.79,0.04
2,15-18,111,2.88,319.32,0.14
3,19-22,231,2.93,676.20,0.30
4,23-26,207,2.94,608.02,0.27
5,27-30,63,2.98,187.99,0.08
6,31-34,46,3.07,141.24,0.06
7,35-38,37,2.81,104.06,0.05
8,39+,20,3.13,62.56,0.03


# Top Spenders

In [9]:
# Group by Screenname 
by_sn = pymoli.groupby(["SN"])

# Find the sum of the price column per screenname
user_spent = pd.DataFrame(by_sn["Price"].sum())
user_count = pd.DataFrame(by_sn["SN"].count())

user_spent.columns = ["Total Spent"]
user_count.columns = ["Total Purchases"]

user_spent.reset_index(inplace=True)
user_count.reset_index(inplace=True)

# Merge the two dfs together
top_five = pd.merge(user_spent,user_count,on="SN")

# Sort the values by Total Spent
top_five = top_five.sort_values("Total Spent",ascending=False)

# Cut the list to the top five
top_five = top_five.head()

# Add a column for Average Purchase
top_five["Average Purchase"] = round((top_five["Total Spent"] / top_five["Total Purchases"]),2)

# Reorder columns, reset index
top5_spenders = top_five[["SN","Total Purchases","Average Purchase","Total Spent"]]
top5_spenders.reset_index(inplace=True,drop=True)

top5_spenders

,SN,Total Purchases,Average Purchase,Total Spent
0,Undirrala66,5,3.41,17.06
1,Saedue76,4,3.39,13.56
2,Mindimnya67,4,3.18,12.74
3,Haellysu29,3,4.24,12.73
4,Eoda93,3,3.86,11.58


# Most Popular Items

In [10]:
# Group by Item Name
by_item = pymoli.groupby(["Item Name"])

# Create dfs with desired metrics to merge together
# Count by Item ID to get Number Sold
number_sold = pd.DataFrame(by_item["Item ID"].count())
number_sold.columns = ["Number Sold"]
number_sold.reset_index(inplace=True)

# Take the average of the price column for Item Price
# (Some items are sold at different prices)
item_price = pd.DataFrame(round(by_item["Price"].mean(),2))
item_price.columns = ["Price (Avg)"]
item_price.reset_index(inplace=True)

# Take the sum of the Price column for the total revenue per item
total_value = pd.DataFrame(by_item["Price"].sum())
total_value.columns = ["Total Value"]
total_value.reset_index(inplace=True)

# Take the max of the Item ID column to pull the Item ID
item_id = pd.DataFrame(by_item["Item ID"].max())
item_id.reset_index(inplace=True)

# Merge the dfs together
merged_byitem = pd.merge(number_sold,item_price,on="Item Name")
merged_byitem = pd.merge(merged_byitem,total_value,on="Item Name")
merged_byitem = pd.merge(merged_byitem,item_id,on="Item Name")

# Reorder columns
merged_byitem = merged_byitem[['Item ID', 'Item Name', 'Number Sold', 'Price (Avg)', 'Total Value']]

# Sort by Number Sold to find most popular items
top5_items = merged_byitem.sort_values("Number Sold",ascending=False)

# Cut to Top Five, reset index
top5_items = top5_items.head()
top5_items.reset_index(inplace=True,drop=True)
top5_items

,Item ID,Item Name,Number Sold,Price (Avg),Total Value
0,101,Final Critic,14,2.76,38.60
1,84,Arcane Gem,11,2.23,24.53
2,39,"Betrayal, Whisper of Grieving Widows",11,2.35,25.85
3,180,Stormcaller,10,3.46,34.65
4,175,Woeful Adamantite Claymore,9,1.24,11.16


# Most Profitable Items

In [11]:
# Re-sort previous df by Total Value to get most profitable games
top5_profits = merged_byitem.sort_values("Total Value",ascending=False)

# Cut to Top Five
top5_profits = top5_profits.head()

top5_profits

,Item ID,Item Name,Number Sold,Price (Avg),Total Value
56,101,Final Critic,14,2.76,38.60
112,34,Retribution Axe,9,4.14,37.26
137,180,Stormcaller,10,3.46,34.65
132,115,Spectral Diamond Doomblade,7,4.25,29.75
96,32,Orenmir,6,4.95,29.70
